## Analysis of cellular interactions with different B cell developmental stages in fetal lung datasets
<br>
<b>Description</b> : In this notebook we run Tangram2 CCC on fetal lung datasets<br>
<b>Author</b> : Hejin Huang (huang.hejin@gene.com)<br>

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import tangram2 as tg2

/gstore/home/huangh83/miniconda3/envs/tangram/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /gstore/home/huangh83/miniconda3/envs/tangram/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Load single-cell data
ad_sc_all = sc.read_h5ad('/gstore/data/resbioai/tangram2_data/lung/single_cell/220715Annotated_Fetal_lung_immune_filtered_soupxcite_cxg.h5ad')
ad_sc_all.X = (np.exp(ad_sc_all.X.toarray()) -1)

# Rank genes and get markers
sc.tl.rank_genes_groups(ad_sc_all, groupby="celltype", use_raw=False)
markers_df = pd.DataFrame(ad_sc_all.uns["rank_genes_groups"]["names"]).iloc[0:100, :]
markers = list(np.unique(markers_df.melt().value.values))

# Filter single-cell data by stage
ad_sc_filtered = ad_sc_all[(ad_sc_all.obs.stage >=12) ].copy()

folder_ID = os.listdir('/gstore/data/resbioai/tangram2_data/lung/spatial_raw/')

In [3]:
ad_map_all = []
composition = []
beta_all = [] # Initialize beta_all here as it's populated in the loop

for sample in folder_ID:
    REAL_DATA_SP_PTH = '/gstore/data/resbioai/tangram2_data/lung/spatial_raw/' + sample 
    ad_sp = ad.read_h5ad(REAL_DATA_SP_PTH)
    
    label_col = 'celltype'
        
    input_dict_1 = tg2.evalkit.met.utils.adatas_to_input({'from': ad_sc_filtered, 'to': ad_sp},
                                                    categorical_labels={'from': [label_col]},
                                                  )

    tg2.evalkit.met.pp.StandardTangram2.run(input_dict_1)

    map_res_1 = tg2.evalkit.met.map_methods.Tangram2Map.run(input_dict_1,
                                                num_epochs = 1000,
                                                genes = markers,
                                             )

    input_dict_1.update(map_res_1)
    
    tg2.evalkit.met.pp.StandardScanpy.run(input_dict_1,target_objs = ['X_from'])

    inter_res = tg2.ccc.TangramCCC.run(input_dict_1,
                                                     n_epochs = 1000,
                                                        learning_rate = 0.01)
    ad_map_all.append(inter_res)
    composition.append(input_dict_1['w'])

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 2212 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Set Solid Seed
Set Solid Seed
Score: 0.793, KL reg: 4.172, Entropy reg: -11.673
Score: 0.869, KL reg: 4.078, Entropy reg: -9.884
Score: 0.870, KL reg: 4.078, Entropy reg: -9.833
Score: 0.870, KL reg: 4.078, Entropy reg: -9.816
Score: 0.870, KL reg: 4.078, Entropy reg: -9.809
Score: 0.870, KL reg: 4.078, Entropy reg: -9.805
Score: 0.870, KL reg: 4.078, Entropy reg: -9.802
Score: 0.870, KL reg: 4.078, Entropy reg: -9.800
Score: 0.870, KL reg: 4.078, Entropy reg: -9.799
Score: 0.870, KL reg: 4.078, Entropy reg: -9.797


INFO:root:Renormalizing Single cell data
INFO:root:Begin training with 2212 genes and rna_count_based density_prior in cells mode after renormalization


Set Solid Seed


INFO:root:Printing scores every 100 epochs.


...

Set Solid Seed


INFO:root:Printing scores every 100 epochs.


Set Solid Seed
Score: 0.892, KL reg: 0.076, Entropy reg: -398674.812
Score: 0.916, KL reg: 0.000, Entropy reg: -381579.688
Score: 0.918, KL reg: 0.000, Entropy reg: -377550.625
Score: 0.918, KL reg: 0.000, Entropy reg: -375014.031
Score: 0.919, KL reg: 0.000, Entropy reg: -372601.125
Score: 0.919, KL reg: 0.000, Entropy reg: -369731.562
Score: 0.919, KL reg: 0.000, Entropy reg: -366339.438
Score: 0.919, KL reg: 0.000, Entropy reg: -362371.156
Score: 0.919, KL reg: 0.000, Entropy reg: -358019.312
Score: 0.919, KL reg: 0.000, Entropy reg: -353357.000


INFO:root:Saving results..
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params
7.084     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
# --- Process and Save Beta Results ---
beta_all = []
for i in range(len(ad_map_all)):
    inter_res = ad_map_all[i]
    beta = inter_res['beta'].to_dataframe()['beta'].copy()
    beta = beta.reset_index()
    beta['inter'] = beta['labels'].astype(str) + '_vs_' + beta['labels_'].astype(str)
    beta.drop(labels = ['labels','labels_',],inplace = True,axis =1)
    beta_all.append(beta)


In [ ]:
# # --- Save Results to CSV ---
# base_output_path = '/gstore/data/resbioai/tangram2_data/telegraph/res/ccc_eval/real/fetal_lung/cci/celltype_raw/'
# for i in range(len(beta_all)):
#     sample_name = folder_ID[i].split('.')[0] # Get sample name from folder_ID
#     beta_all[i].to_csv(base_output_path + sample_name + '.csv')
#     composition[i].to_csv(base_output_path + sample_name + '_composition.csv')